Importing basic libraries

In [15]:
import numpy as np
import pandas as pd

Fetch the processed file (this is the file which we saved after processing the original dataset)

In [16]:
df= pd.read_csv('https://github.com/coder-rakeshpaul/EmotionAnalysis/raw/main/main/saved%20data/processed%20data.csv')
df.to_csv('text.txt' , columns= ['text'] , header = False , index = False)

In [17]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Generating vector representations for the text using fasttext

In [18]:
import fasttext

un_model = fasttext.train_unsupervised('text.txt')

adding a column in the dataframe of vectorized representation of the input text

In [19]:
df.text = df.text.astype(str)
inputs = []
for texts in df['text']:
    inputs.append(un_model.get_sentence_vector(texts))

df['Inputs'] = inputs

Since the data is imbalanced we apply SMOTE to oversample the minority classes

In [20]:
from imblearn.over_sampling import SMOTE

X = np.array([input for input in df['Inputs']])
y = np.array(df['Emotions'])

smt = SMOTE(random_state=42)
X, y = smt.fit_resample(X, y)

spliting the data into train and test

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, shuffle=True)

training the data with Random Forest Classifier

In [22]:
from sklearn.ensemble import RandomForestClassifier


RFC = RandomForestClassifier(n_estimators = 100)  
RFC.fit(X_train, y_train)

RandomForestClassifier()

Getting the accuracy score and the f1 score of the model

In [23]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

y_pred = RFC.predict(X_test)
accuracy= accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred , average=None)

print(accuracy)
print(f1)

0.9612498845692123
[0.92283759 0.96336241 0.96646389 0.95807605 0.93916106 0.96342607
 0.96851911 0.96196243 0.98210291 0.98012821 0.94632768 0.98945687
 0.99523356 0.97876956 0.99344262 0.96379197 0.9978664  0.9765438
 0.96539514 0.99657255 0.39575099 0.97157458 0.99777283 0.98198483
 0.99819346 0.99439372 0.98904588 0.98030566]


Saving the model

In [24]:
import pickle
with open('model_pkl', 'wb') as files:
    pickle.dump(RFC, files)